In [ ]:
# display plots inline
%matplotlib inline 

# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline

# load dataset
df = pd.read_csv("stroke_data.csv")

# check for missing values
msno.bar(df)
fig = plt.figure(figsize=(25, 20))
sns.heatmap(df.corr(), annot=True)
plt.savefig("Heatmap.png")

In [35]:
# check the dataset variables and summary statistics
df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   object 
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   object 
 5   work_type          4981 non-null   object 
 6   Residence_type     4981 non-null   object 
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   object 
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 428.2+ KB


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [ ]:
# look for response categories
print(df['gender'].value_counts())

# convert string data to numerical
def gender_to_numeric(x):
        if x=='Female': return 0
        if x=='Male':   return 1

df['gender'] = df['gender'].apply(gender_to_numeric)

In [ ]:
# look for response categories
print(df['ever_married'].value_counts())

# convert string data to numerical
def ever_married_to_numeric(x):
        if x=='Yes': return 1
        if x=='No':   return 0

df['ever_married'] = df['ever_married'].apply(ever_married_to_numeric)

In [ ]:
# look for response categories
print(df['Residence_type'].value_counts())

# convert string data to numerical
def Residence_type_to_numeric(x):
        if x=='Urban': return 1
        if x=='Rural':   return 0

df['Residence_type'] = df['Residence_type'].apply(Residence_type_to_numeric)

In [ ]:
# look for response categories
print(df['work_type'].value_counts())

# hot encoder for multiple response categories
encoder=ce.OneHotEncoder(cols='work_type',handle_unknown='return_nan',return_df=True,use_cat_names=True)
df = encoder.fit_transform(df)

In [ ]:
# look for response categories
print(df['smoking_status'].value_counts())

# hot encoder for multiple response categories
encoder=ce.OneHotEncoder(cols='smoking_status',handle_unknown='return_nan',return_df=True,use_cat_names=True)
df = encoder.fit_transform(df)

In [ ]:
# create Numpy array for dependent variable y
y = df['stroke'].values

# create Numpy 2-D array for independent variables X
df_temp = df.copy(deep=True)
df_temp.drop('stroke', inplace=True, axis=1)
X = df_temp.values

In [ ]:
# standardize independent variables X
std_scl = StandardScaler()
std_scl.fit_transform(X)

In [ ]:
# create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=2)


# initialize Logistic Regression
model = LogisticRegression(solver='liblinear')

# fit Logistic Regression
model.fit(X_train, y_train)

# predict y values
y_pred = model.predict(X_test)
print("prediction for datapoint 0:", model.predict([X[0]]))

# display model score for test data (correct prediction perecentage)
print(model.score(X_test, y_test))

In [ ]:
# create confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix(y_test, y_pred))

# visualize confusion matrix
sns.heatmap(cf_matrix, annot=True, cmap='Blues')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)

# display accuracy, precision, recall, f1 score
print("accuracy:", accuracy_score(y_test, y_pred))
print("precision:", precision_score(y_test, y_pred))
print("recall:", recall_score(y_test, y_pred))
print("f1 score:", f1_score(y_test, y_pred))

In [ ]:
# check predicted probabilites
model.predict_proba(X_test)
model.predict_proba(X_test)[:, 1]

# compute predictions for a different cut-off (0.10)
y_pred_10 = model.predict_proba(X_test)[:, 1] > 0.1
cf_matrix_10 = confusion_matrix(y_test, y_pred_10)

# display confusion matrix
print("\nTest confusion_matrix")
sns.heatmap(cf_matrix_10, annot=True, cmap='Blues')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)

In [ ]:
# compute false positive rate, true positive rates for different cut-off points
y_pred_prob = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])

# display ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('1 - specificity')
plt.ylabel('sensitivity')
plt.show()

In [ ]:
# compute ROC AUC score
print("model 1 AUC score:", roc_auc_score(y_test, y_pred_prob[:, 1]))

# compute ROC AUC score for a restrictec model (with only two independent variables)
model2 = LogisticRegression()
model2.fit(X_train[:, 0:2], y_train)
y_pred_proba3 = model2.predict_proba(X_test[:, 0:2])
print("model 2 AUC score:", roc_auc_score(y_test, y_pred_proba3[:, 1]))

In [37]:
# initialize K-fold
kf = KFold(n_splits=5, shuffle=True)

# create new variable columns
df['male'] = df['gender'] == 'Male'
df['smokes'] = df['smoking_status'] == 'smokes'
df['urban'] = df['Residence_type'] == 'Urban'
df['work_in_private'] = df['work_type'] == 'Private'
df['ever_married'] = df['ever_married'] == 'Yes'

# create numpy 2-D arrays for different models
X1 = df[['male', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi','smokes', 'urban', 'work_in_private', 'ever_married']].values
X2 = df[['male', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi','smokes', 'urban']].values
X3 = df[['male', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level']].values
y = df['stroke'].values

# define function two compute means of accuracy, precision, recall, f1 score for different k-fold splits
def score_model(X, y, kf):
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = LogisticRegression(max_iter=300)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
    print("accuracy:", np.mean(accuracy_scores))
    print("precision:", np.mean(precision_scores))
    print("recall:", np.mean(recall_scores))
    print("f1 score:", np.mean(f1_scores))

# fit model and display mean model scores for each model
print("Logistic Regression for Model 1")
score_model(X1, y, kf)
print()
print("Logistic Regression for Model 2")
score_model(X2, y, kf)
print()
print("Logistic Regression for Model 3")
score_model(X3, y, kf)

Logistic Regression for Model 1


/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.9502100679548686
precision: 0.2
recall: 0.004081632653061224
f1 score: 0.007999999999999998

Logistic Regression for Model 2


/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: Un

accuracy: 0.9504104683528498
precision: 0.2
recall: 0.0047619047619047615
f1 score: 0.009302325581395349

Logistic Regression for Model 3


/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.950210672177174
precision: 0.0
recall: 0.0
f1 score: 0.0


/home/nimnathw/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# apply PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=None)
pca.fit(X)

# display eigenvalues
print("Eigenvalues:")
print(pca.explained_variance_)
print()

# display explained variances
print("Variances (Percentage):")
print(pca.explained_variance_ratio_ * 100)
print()

# create scree plot
plt.plot(np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel("Number of dimensions)")
plt.ylabel("Explained variance (%)")

In [ ]:
# apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# create transformed dataset
X_pca = pd.DataFrame(X_pca)
print(X_pca.head())
print("\nSize: ")
print(X_pca.shape)

# display pca correlation heatmap
fig = plt.figure(figsize=(10, 8))
sns.heatmap(X_pca.corr(), annot=True)

In [ ]:
# create train and test sets
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.30, shuffle=True, random_state=2)

# initialize logistic regression model
model_pca = LogisticRegression(max_iter=2500)

# train model
model_pca.fit(X_train_pca, y_train)

# compute predictions
y_pred_pca = model_pca.predict(X_test_pca) # Predictions

# measure accuracy
print("Train accuracy:", np.round(accuracy_score(y_train, model_pca.predict(X_train_pca)), 2))
print("Test accuracy:", np.round(accuracy_score(y_test, y_pred_pca), 2))

# create confusion matrix
from sklearn.metrics import confusion_matrix
cf_matrix_pca = confusion_matrix(y_test, y_pred_pca)
print("\nTest confusion_matrix")
sns.heatmap(cf_matrix_pca, annot=True, cmap='Blues')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)

In [ ]:
# construct pipeline for model with PCA
sc = StandardScaler()
pca = PCA(n_components=2)
log_reg = LogisticRegression(max_iter=2500)

log_reg_model = Pipeline([
    ('std_scaler', sc),
    ('pca', pca),
    ('regressor', log_reg)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=2)
log_reg_model.fit(X_train,y_train)

# compute predictions
y_pred_log_reg = log_reg_model.predict(X_test)

# create confusion matrix
cf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)

# display confusion matrix
print("\nTest confusion_matrix")
sns.heatmap(cf_matrix_log_reg, annot=True, cmap='Blues')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)

# display prescion recall fscore
print(precision_recall_fscore_support(y_test, y_pred_log_reg))